# Embeddings  [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1VGqw8yMPsFmlH5IHv8Vca31OVaikcgrh?usp=sharing)

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)


Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстах. Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


### RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

### Визуализация
А [вот тут](https://projector.tensorflow.org/) есть хорошая визуализация для английского.

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [2]:
!pip install gensim

     |████████████████████████████████| 24.0 MB 3.8 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 5.4 MB/s eta 0:00:011
You should consider upgrading via the '/Users/ktulhu/maryszmary_ve/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install bs4

     |████████████████████████████████| 97 kB 2.1 MB/s eta 0:00:011
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=c0309604d5a40f94ad16575ce7fe562f4a49df661d4c9c79a52f9328d3c41b0e
  Stored in directory: /Users/ktulhu/Library/Caches/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
You should consider upgrading via the '/Users/ktulhu/maryszmary_ve/bin/python -m pip install --upgrade pip' command.


In [5]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/ktulhu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x104536430>)

In [7]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

Возьмем несколько слов для примера:

In [8]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [9]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297018885612488
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.555358350276947
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  

Находим косинусную близость пары слов:

In [10]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.23895611


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [11]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


In [12]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

'ананас_S'

**Упражнения для разминки**

Найдите пример многозначного слова, для которого в топ-10 (метод `most_similar`) похожих на него слов входят слова связанные с разными значениями:

По аналогии с Италия -- пицца, Сибирь -- пельмень, придумайте похожую связку слов для проверки: 

Приведите пример трех слов w1, w2, w3, таких, что w1 и w2 являются синонимами, w1 и w3 являются антонимами, но при этом, similarity(w1, w2) < similarity(w1, w3).

### Задание

Напишите функцию, которая принимает на вход предложение, и заменяет случайное существительное в нём на "ассоциат" -- ближайшее к нему слово из модели word2vec.

NB: для этого вам понадобится морфологический анализатор. Советую использовать pymorphy (мы кратко говорили про него на прошлом семинаре).

 как пользоваться pymorphy:

In [13]:
!pip install pymorphy2

You should consider upgrading via the '/Users/ktulhu/maryszmary_ve/bin/python -m pip install --upgrade pip' command.


In [14]:
from pymorphy2 import MorphAnalyzer

In [15]:
analyser = MorphAnalyzer()

In [16]:
# разобрать слово (в данном случае возможно два разбора, поэтому получаем список из двух элементов)
result = analyser.parse('слово')
result

[Parse(word='слово', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='слово', score=0.59813, methods_stack=((DictionaryAnalyzer(), 'слово', 54, 0),)),
 Parse(word='слово', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='слово', score=0.401869, methods_stack=((DictionaryAnalyzer(), 'слово', 54, 3),))]

In [ ]:
# достать часть речи
result[0].tag.POS

'NOUN'

In [17]:
# поставить в дательный падеж
result[0].inflect(frozenset(['datv'])).word

'слову'

Ваша функция (для простоты можно не пытаться поставить слово в "нужную" форму и ограничиться именительным падежом):

In [ ]:
def change_random_noun(sentence):
    pass

## Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [18]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2021-10-15 13:39:46--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 67281491 (64M) [text/plain]
Сохранение в: «unlabeledTrainData.tsv»

unlabeledTrainData. 100%[===================>]  64.16M  7.41MB/s    за 9.1s    

2021-10-15 13:39:58 (7.07 MB/s) - «unlabeledTrainData.tsv» сохранён [67281491/67281491]



In [19]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [20]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [21]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [30]:
def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки
    review_text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
#     review_text = BeautifulSoup(review_text, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    words = review_text.lower().split()
    if remove_stopwords: # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    # разбиваем обзор на предложения
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    # применяем предыдущую функцию к каждому предложению
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [33]:
from tqdm import tqdm

In [34]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in tqdm(data["review"]):
    sentences += review_to_sentences(review, tokenizer)

  0%|          | 192/50000 [00:00<00:26, 1913.04it/s]

Parsing sentences from training set...


100%|██████████| 50000/50000 [00:23<00:00, 2111.93it/s]


In [35]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [36]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [38]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, vector_size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 2min 23s, sys: 565 ms, total: 2min 24s
Wall time: 38 s


Смотрим, сколько в модели слов.

In [45]:
print(len(model_en.wv.key_to_index))

28316


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [46]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7792311310768127)]
[('men', 0.6633631587028503)]
[('europe', 0.7178550958633423), ('canada', 0.6989084482192993), ('australia', 0.6931552886962891)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [47]:
model_en.wv.similarity('lion', 'rabbit')

0.2506585

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [48]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2021-10-15 13:47:49--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 167631 (164K) [text/plain]
Сохранение в: «alice.txt»

alice.txt           100%[===================>] 163.70K  --.-KB/s    за 0.1s    

2021-10-15 13:47:50 (1.45 MB/s) - «alice.txt» сохранён [167631/167631]



In [49]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст
text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

# убираем всю пунктуацию и делим текст на слова по пробелу
punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [50]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [53]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(97075, 150225)

Лев и кролик стали чуть ближе друг к другу!

In [55]:
model.wv.similarity('lion', 'rabbit')

0.25955093

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [ ]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores.

In [58]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2021-10-15 13:50:10--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 871776 (851K) [text/plain]
Сохранение в: «ru_analogy_tagged.txt.1»

ru_analogy_tagged.t 100%[===================>] 851.34K  4.62MB/s    за 0.2s    

2021-10-15 13:50:10 (4.62 MB/s) - «ru_analogy_tagged.txt.1» сохранён [871776/871776]



In [59]:
!head ru_analogy_tagged.txt

: capital-common-countries
афины_S греция_S багдад_S ирак_S
афины_S греция_S бангкок_S таиланд_S
афины_S греция_S пекин_S китай_S
афины_S греция_S берлин_S германия_S
афины_S греция_S берн_S швейцария_S
афины_S греция_S каир_S египет_S
афины_S греция_S канберра_S австралия_S
афины_S греция_S ханой_S вьетнам_S
афины_S греция_S гавана_S куба_S


## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [72]:
! git clone https://github.com/facebookresearch/fastText.git
! pip3 install fastText/.

Клонирование в «fastText»…
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Получение объектов: 100% (3854/3854), 8.22 MiB | 938.00 KiB/s, готово.
Определение изменений: 100% (2417/2417), готово.
Processing ./fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp38-cp38-macosx_10_14_x86_64.whl size=344360 sha256=005bcadd1cc6df9a6f8d8c6326dc788793fc50eb5a825f79d0257b2d7e38dcdc
  Stored in directory: /private/var/folders/zy/214v8d1d4tng0_jtxf69crf40000gp/T/pip-ephem-wheel-cache-zd8oz4ct/wheels/f0/7c/da/fe73eff2b10eada58ea862d143fc14e2197ecb05a3545ae5ac
Successfully built fasttext
You should consider upgrading via the '/Users/ktulhu/maryszmary_ve/bin/python -m pip install --upgrade pip' command.


In [73]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [74]:
ft_model.get_word_vector("movie")[:10]

array([-0.1313279 , -0.02241677, -0.18896143, -0.06513993, -0.09668837,
        0.06699901, -0.17133492, -0.08639518, -0.1176882 ,  0.04813271],
      dtype=float32)

In [75]:
ft_model.get_nearest_neighbors('actor')

[(0.9998166561126709, 'actors'),
 (0.9997903108596802, 'actresses'),
 (0.9996681213378906, 'presence'),
 (0.9996576309204102, 'perfect'),
 (0.9996448755264282, 'act'),
 (0.9996447563171387, 'actual'),
 (0.9996442794799805, 'director'),
 (0.9996076226234436, 'influence'),
 (0.9996068477630615, 'prince'),
 (0.9995878338813782, 'violence')]

In [83]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.9996215105056763, 'actor'),
 (0.999507486820221, 'act'),
 (0.9994754791259766, 'acts'),
 (0.9994434118270874, 'actors'),
 (0.9993497133255005, 'actual'),
 (0.9992961287498474, 'actresses'),
 (0.9992280006408691, 'actions'),
 (0.9992159008979797, 'perfect'),
 (0.9991544485092163, 'director'),
 (0.9991104602813721, 'perfectly')]

In [84]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.9996706247329712, 'reviews'),
 (0.9996266961097717, 'just'),
 (0.9996209144592285, 'movie'),
 (0.9996070265769958, 'watchable'),
 (0.999596357345581, 'move'),
 (0.9995846748352051, 'probably'),
 (0.9995819330215454, 'review'),
 (0.9995530843734741, 'views'),
 (0.9995368123054504, 'rented'),
 (0.9995203614234924, 'realise')]

In [85]:
df = pd.read_csv('../sem1/tweets_sentiment.csv')

In [88]:
df.head()

,text,label
0,мыс на меня обиделась:(\nя ей даже ничего не с...,negative
1,"аааааааааааааааааааа,не хочу на работу :(",negative
2,"У меня какой-то особенный вид ушей! :D, некото...",positive
3,@simonovkon он неплохой человек в жизни. Я ра...,negative
4,"RT @Darina_Lo: Домааааа\nЕхали на такси, пели ...",positive


In [89]:
len(df)

226834

Запишем полученные данные в формате для обучения классификатора:

In [91]:
import numpy as np

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
X = df.text.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [95]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [96]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8373944640376189
R@1: 0.8373944640376189
Number of examples: 74856


## Задание

1. Мы будем работать с данными fakenews отсюда: https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации.
3. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста. 
4. Обучите на полученных средних векторах алгоритм классификации.

Бонус: Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.

In [40]:
!wget https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv

--2021-10-15 13:46:15--  https://raw.githubusercontent.com/diptamath/covid_fake_news/main/data/Constraint_Train.csv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 1253562 (1.2M) [text/plain]
Сохранение в: «Constraint_Train.csv»

Constraint_Train.cs 100%[===================>]   1.20M  5.16MB/s    за 0.2s    

2021-10-15 13:46:15 (5.16 MB/s) - «Constraint_Train.csv» сохранён [1253562/1253562]



In [41]:
df = pd.read_csv('Constraint_Train.csv')

In [43]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real
